In [1]:
#iniciar spark
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
#conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c36c9d14-be54-4435-a1dd-feb6c4360638;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 434ms :: artifacts dl 31ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------------------------

In [2]:
df = spark.read.option('delimiter', ';').option('header', 'true').option("encoding","ISO-8859-1").csv('s3a://catarina-3sprint-sptech-bucket-bruto/controle_mensal_demais_parametros.csv')

22/06/15 17:26:16 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:
df.show()


22/06/15 17:26:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------------+---+-----------------+--------------------+-----------+-------------------+--------------------+--------------------+-------------------+---------------------------------+---------------------------------+------------------------------+-----------------------------+------------------------------+--------------------+-----------------+-----------------+--------------------+-----------------------------------------+--------------------+----------------+----------------------------------+-----------------------------+------------------------------------------+-------------------------------------+----------------+------------+---------+
|Região Geográfica| UF|Regional de Saúde|           Município|Código IBGE|Tipo da Instituição|Sigla da Instituição| Nome da Instituição|CNPJ da Instituição|Nome do escritório regional/local|CNPJ do escritório regional/local|Tipo da Forma de Abastecimento|Código Forma de abastecimento|Nome da Forma de Abastecimento|   Nome da ETA / UTA|An

In [4]:
# Limpando a tabela
dfMensal= df.drop('Código IBGE',' Município','Sigla da Instituição','Código Forma de abastecimento',
                  'CNPJ da Instituição','Nome do escritório regional/local','CNPJ do escritório regional/local',
                  ' Nome da ETA / UTA','Regional de Saúde','ipo da Forma de Abastecimento','Nome da Forma de Abastecimento',
                  'Nome da ETA','Regional de Saúde','Nome da ETA / UTA','Tipo da Forma de Abastecimento','Tipo da Instituição',
                  'Município','Data de preenchimento do relatório mensal','Tipo de captação',
                 'categoria do manancial superficial','Categoria do ponto de captação subterrânea', 'Nome do ponto de captação subterrânea',
                 '   Data de registro ','Data da coleta', 'Nome do manancial superficial','Unidade')
dfMensal.show(5)

+-----------------+---+--------------------+-----------------+-----------------+--------------------+----------------+---------+
|Região Geográfica| UF| Nome da Instituição|Ano de referência|Mês de referência|   Data de registro |       Parâmetro|Resultado|
+-----------------+---+--------------------+-----------------+-----------------+--------------------+----------------+---------+
|          SUDESTE| MG|COMPANHIA DE SANE...|             2014|                8|2014/09/25 00:00:...|Escherichia coli|    214,1|
|          SUDESTE| MG|COMPANHIA DE SANE...|             2015|                2|2015/03/25 00:00:...|Escherichia coli|      1,0|
|          SUDESTE| MG|COMPANHIA DE SANE...|             2015|                3|2015/04/29 00:00:...|Escherichia coli|    310,0|
|          SUDESTE| MG|COMPANHIA DE SANE...|             2015|                4|2015/05/29 00:00:...|Escherichia coli|    410,0|
|            NORTE| PA|COMPANHIA DE SANE...|             2021|                4|2021/05/04 00:00:

In [55]:
dfMensal.withColumnRenamed("Região Geográfica","Regiao").withColumnRenamed("Mês de referência","mes").withColumnRenamed("Ano de referência","ano").withColumnRenamed("Parâmetro","parametro").show(1000)

+--------+---+--------------------+----+---+--------------------+----------------+---------+
|  Regiao| UF| Nome da Instituição| ano|mes|   Data de registro |       parametro|Resultado|
+--------+---+--------------------+----+---+--------------------+----------------+---------+
| SUDESTE| MG|COMPANHIA DE SANE...|2014|  8|2014/09/25 00:00:...|Escherichia coli|    214,1|
| SUDESTE| MG|COMPANHIA DE SANE...|2015|  2|2015/03/25 00:00:...|Escherichia coli|      1,0|
| SUDESTE| MG|COMPANHIA DE SANE...|2015|  3|2015/04/29 00:00:...|Escherichia coli|    310,0|
| SUDESTE| MG|COMPANHIA DE SANE...|2015|  4|2015/05/29 00:00:...|Escherichia coli|    410,0|
|   NORTE| PA|COMPANHIA DE SANE...|2021|  4|2021/05/04 00:00:...|Escherichia coli| AUSÊNCIA|
|   NORTE| PA|COMPANHIA DE SANE...|2020|  2|2020/09/28 00:00:...|Escherichia coli|  AUSENTE|
| SUDESTE| MG|COMPANHIA DE SANE...|2016|  4|2016/08/08 00:00:...|Escherichia coli|  ausente|
|   NORTE| PA|COMPANHIA DE SANE...|2017| 12|2018/02/15 00:00:...|Esche

In [56]:
dfMensal.show(vertical=True)

-RECORD 0-----------------------------------
 Região Geográfica   | SUDESTE              
 UF                  | MG                   
 Nome da Instituição | COMPANHIA DE SANE... 
 Ano de referência   | 2014                 
 Mês de referência   | 8                    
 Data de registro    | 2014/09/25 00:00:... 
 Parâmetro           | Escherichia coli     
 Resultado           | 214,1                
-RECORD 1-----------------------------------
 Região Geográfica   | SUDESTE              
 UF                  | MG                   
 Nome da Instituição | COMPANHIA DE SANE... 
 Ano de referência   | 2015                 
 Mês de referência   | 2                    
 Data de registro    | 2015/03/25 00:00:... 
 Parâmetro           | Escherichia coli     
 Resultado           | 1,0                  
-RECORD 2-----------------------------------
 Região Geográfica   | SUDESTE              
 UF                  | MG                   
 Nome da Instituição | COMPANHIA DE SANE... 
 Ano de re

In [33]:
#UPANDO O 1° TRATAMENTO

#dfMensal.write.mode('overwrite').csv('s3a://catarina-3sprint-sptech-bucket-tratados/mensal')

22/06/15 16:35:55 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/15 16:35:56 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/15 16:35:56 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/15 16:36:04 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/15 16:36:04 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/15 16:36:10 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/15 16:36:10 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [54]:
dfEscherichia= dfMensal.select('UF','Parâmetro','Resultado').where(dfMensal['Parâmetro'] == 'Escherichia coli')

In [53]:
dfEscherichia.show(10)

+---+----------------+---------+
| UF|       Parâmetro|Resultado|
+---+----------------+---------+
| MG|Escherichia coli|    214,1|
| MG|Escherichia coli|      1,0|
| MG|Escherichia coli|    310,0|
| MG|Escherichia coli|    410,0|
| PA|Escherichia coli| AUSÊNCIA|
| PA|Escherichia coli|  AUSENTE|
| MG|Escherichia coli|  ausente|
| PA|Escherichia coli|        P|
| MG|Escherichia coli|  Ausente|
| MG|Escherichia coli|        0|
+---+----------------+---------+
only showing top 10 rows



In [29]:
dfCrypto = dfMensal.select('UF','Parâmetro','Resultado').where(dfMensal['Parâmetro'] == 'Cryptosporidium')

In [30]:
dfCrypto.show()

+---+---------------+---------+
| UF|      Parâmetro|Resultado|
+---+---------------+---------+
| CE|Cryptosporidium|      ,01|
| MG|Cryptosporidium|        0|
| CE|Cryptosporidium|        0|
| CE|Cryptosporidium|        0|
| PR|Cryptosporidium|        0|
| PR|Cryptosporidium|     0,00|
| PR|Cryptosporidium|        0|
| PR|Cryptosporidium|        0|
| PR|Cryptosporidium|        0|
| PR|Cryptosporidium|        0|
| PR|Cryptosporidium|        0|
| PR|Cryptosporidium|        0|
| MG|Cryptosporidium|        0|
| MG|Cryptosporidium|        0|
| MG|Cryptosporidium|        0|
| MG|Cryptosporidium|        0|
| MG|Cryptosporidium|        0|
| MG|Cryptosporidium|        0|
| MG|Cryptosporidium|        0|
| MG|Cryptosporidium|      1,2|
+---+---------------+---------+
only showing top 20 rows



In [31]:
dfVirus = dfMensal.select('UF','Parâmetro','Resultado').where(dfMensal['Parâmetro'] == 'Vírus')

In [46]:
dfVirus.show()

+---+---------+---------+
| UF|Parâmetro|Resultado|
+---+---------+---------+
| CE|    Vírus|      ,01|
| MG|    Vírus|        0|
| MG|    Vírus|        0|
| MG|    Vírus|        0|
| MG|    Vírus|        0|
| MG|    Vírus|        0|
| MG|    Vírus|        0|
| MG|    Vírus|        0|
| MG|    Vírus|        0|
| MG|    Vírus|        0|
| PR|    Vírus|      100|
| PR|    Vírus|        0|
| PR|    Vírus|        0|
| PR|    Vírus|        0|
| PR|    Vírus|        0|
| PR|    Vírus|        0|
| PR|    Vírus|        0|
| MG|    Vírus|        0|
| MG|    Vírus|        0|
| MG|    Vírus|    214,3|
+---+---------+---------+
only showing top 20 rows



In [32]:
dfClorofilia = dfMensal.select('UF','Parâmetro','Resultado').where(dfMensal['Parâmetro'] == 'Clorofila-a')

In [33]:
dfClorofilia.show()

+---+-----------+---------+
| UF|  Parâmetro|Resultado|
+---+-----------+---------+
| MG|Clorofila-a|      1,2|
| CE|Clorofila-a|    40,83|
| CE|Clorofila-a|    33,97|
| CE|Clorofila-a|    36,48|
| CE|Clorofila-a| 0,034799|
| CE|Clorofila-a| 0,027768|
| CE|Clorofila-a|     5,56|
| CE|Clorofila-a|    7,383|
| CE|Clorofila-a|     1,21|
| CE|Clorofila-a|     0,97|
| CE|Clorofila-a|      3,0|
| CE|Clorofila-a|  0,01713|
| CE|Clorofila-a|  0,02297|
| CE|Clorofila-a|     3,99|
| CE|Clorofila-a|    87,27|
| CE|Clorofila-a|      3,5|
| CE|Clorofila-a| 0,001226|
| CE|Clorofila-a|    91,44|
| CE|Clorofila-a|     31,6|
| CE|Clorofila-a| 0,002376|
+---+-----------+---------+
only showing top 20 rows



In [35]:
dfUm = dfVirus.join(dfClorofilia,dfVirus["UF"] == dfClorofilia["UF"])

In [36]:
dfUm.show(10)

+---+---------+---------+---+-----------+---------+
| UF|Parâmetro|Resultado| UF|  Parâmetro|Resultado|
+---+---------+---------+---+-----------+---------+
| AL|    Vírus|        0| AL|Clorofila-a|     1,03|
| AL|    Vírus|        0| AL|Clorofila-a|        0|
| AL|    Vírus|        0| AL|Clorofila-a|        0|
| AL|    Vírus|        0| AL|Clorofila-a|        0|
| AL|    Vírus|        0| AL|Clorofila-a|        0|
| AL|    Vírus|        0| AL|Clorofila-a|      2,5|
| AL|    Vírus|        0| AL|Clorofila-a|        0|
| AL|    Vírus|        0| AL|Clorofila-a|        0|
| AL|    Vírus|        0| AL|Clorofila-a|      1,6|
| AL|    Vírus|        0| AL|Clorofila-a|     0,96|
+---+---------+---------+---+-----------+---------+
only showing top 10 rows



In [37]:
dfDois = dfCrypto.join(dfEscherichia,dfCrypto["UF"] == dfEscherichia["UF"])

In [38]:
dfDois.show()

+---+---------------+---------+---+----------------+---------+
| UF|      Parâmetro|Resultado| UF|       Parâmetro|Resultado|
+---+---------------+---------+---+----------------+---------+
| ES|Cryptosporidium|        0| ES|Escherichia coli|      120|
| ES|Cryptosporidium|        0| ES|Escherichia coli|       <1|
| ES|Cryptosporidium|        0| ES|Escherichia coli|       <1|
| ES|Cryptosporidium|        0| ES|Escherichia coli|      184|
| ES|Cryptosporidium|        0| ES|Escherichia coli|       <1|
| ES|Cryptosporidium|        0| ES|Escherichia coli|    285,1|
| ES|Cryptosporidium|        0| ES|Escherichia coli|       <1|
| ES|Cryptosporidium|        0| ES|Escherichia coli|       <1|
| ES|Cryptosporidium|        0| ES|Escherichia coli|       <1|
| ES|Cryptosporidium|        0| ES|Escherichia coli|       <1|
| ES|Cryptosporidium|        0| ES|Escherichia coli|       <1|
| ES|Cryptosporidium|        0| ES|Escherichia coli|       <1|
| ES|Cryptosporidium|        0| ES|Escherichia coli|   

In [61]:
dataframe2 = dfDois.drop('Parâmetro')
dataframe2.show()

+---+---------+---+---------+
| UF|Resultado| UF|Resultado|
+---+---------+---+---------+
| ES|        0| ES|      120|
| ES|        0| ES|       <1|
| ES|        0| ES|       <1|
| ES|        0| ES|      184|
| ES|        0| ES|       <1|
| ES|        0| ES|    285,1|
| ES|        0| ES|       <1|
| ES|        0| ES|       <1|
| ES|        0| ES|       <1|
| ES|        0| ES|       <1|
| ES|        0| ES|       <1|
| ES|        0| ES|       <1|
| ES|        0| ES|    24196|
| ES|        0| ES|      0,0|
| ES|        0| ES|      0,0|
| ES|        0| ES|      0,0|
| ES|        0| ES|      214|
| ES|        0| ES|        0|
| ES|        0| ES|       46|
| ES|        0| ES|      387|
+---+---------+---+---------+
only showing top 20 rows



In [62]:
dataframe1 = dfUm.drop('Parâmetro')
dataframe1.show()

+---+---------+---+---------+
| UF|Resultado| UF|Resultado|
+---+---------+---+---------+
| AL|        0| AL|     1,03|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|      2,5|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|      1,6|
| AL|        0| AL|     0,96|
| AL|        0| AL|     1,08|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|        0|
| AL|        0| AL|     0,96|
+---+---------+---+---------+
only showing top 20 rows



In [65]:
#dfFinal = dataframe1.join(dataframe2,dataframe1["UF"] == dataframe2["UF"])

In [67]:
unionDF = dataframe1.union(dataframe2)
#unionDF.show(truncate=False)

In [ ]:
#unionDF .coalesce(1).write.option("header", True).csv("s3a://catarina-3sprint-sptech-bucket-tratados/randons", emptyValue='')
